In [1]:
#Import Libraries
import glob
from keras.models import Sequential, load_model
import numpy as np
import pandas as pd
import keras as k
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import  accuracy_score, f1_score, precision_score,confusion_matrix, recall_score, roc_auc_score
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.svm import SVC
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import Image

In [2]:
df = pd.read_csv("dataset/kidney_disease.csv")
df.head()

,id,age,bp,sg,al,su,rbc,pc,pcc,ba,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [3]:
df.shape

(400, 26)

In [4]:
#Create a list of columns to retain
columns_to_retain = ["sg", "al", "sc", "hemo",
                         "pcv", "wbcc", "rbcc", "htn", "classification"]

#columns_to_retain = df.columns, Drop the columns that are not in columns_to_retain
df = df.drop([col for col in df.columns if not col in columns_to_retain], axis=1)
    
# Drop the rows with na or missing values
df = df.dropna(axis=0)

In [5]:
df

,sg,al,sc,hemo,pcv,htn,classification
0,1.020,1.0,1.2,15.4,44,yes,ckd
1,1.020,4.0,0.8,11.3,38,no,ckd
2,1.010,2.0,1.8,9.6,31,no,ckd
3,1.005,4.0,3.8,11.2,32,yes,ckd
4,1.010,2.0,1.4,11.6,35,no,ckd
...,...,...,...,...,...,...,...
395,1.020,0.0,0.5,15.7,47,no,notckd
396,1.025,0.0,1.2,16.5,54,no,notckd
397,1.020,0.0,0.6,15.8,49,no,notckd
398,1.025,0.0,1.0,14.2,51,no,notckd


In [6]:
for column in df.columns:
        if df[column].dtype == np.number:
            continue
        df[column] = LabelEncoder().fit_transform(df[column])

In [7]:
df

,sg,al,sc,hemo,pcv,htn,classification
0,1.020,1.0,1.2,15.4,28,1,0
1,1.020,4.0,0.8,11.3,22,0,0
2,1.010,2.0,1.8,9.6,15,0,0
3,1.005,4.0,3.8,11.2,16,1,0
4,1.010,2.0,1.4,11.6,19,0,0
...,...,...,...,...,...,...,...
395,1.020,0.0,0.5,15.7,31,0,1
396,1.025,0.0,1.2,16.5,38,0,1
397,1.020,0.0,0.6,15.8,33,0,1
398,1.025,0.0,1.0,14.2,35,0,1


In [8]:
#Split the data
X = df.drop(["classification"], axis=1)
y = df["classification"]

In [9]:
X

,sg,al,sc,hemo,pcv,htn
0,1.020,1.0,1.2,15.4,28,1
1,1.020,4.0,0.8,11.3,22,0
2,1.010,2.0,1.8,9.6,15,0
3,1.005,4.0,3.8,11.2,16,1
4,1.010,2.0,1.4,11.6,19,0
...,...,...,...,...,...,...
395,1.020,0.0,0.5,15.7,31,0
396,1.025,0.0,1.2,16.5,38,0
397,1.020,0.0,0.6,15.8,33,0
398,1.025,0.0,1.0,14.2,35,0


In [10]:
y

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    1
399    1
Name: classification, Length: 287, dtype: int32

In [12]:
#Split the data into 80% training and 20% testing 
X_train,  X_test, y_train, y_test = train_test_split(
        X, y, test_size= 0.2, shuffle=True)

In [13]:
X_train

,sg,al,sc,hemo,pcv,htn
62,1.020,3.0,0.6,11.0,17,1
305,1.020,0.0,0.8,17.1,25,0
4,1.010,2.0,1.4,11.6,19,0
293,1.020,0.0,1.2,14.9,32,0
12,1.015,3.0,2.1,9.7,12,1
...,...,...,...,...,...,...
347,1.025,0.0,1.0,17.8,27,0
257,1.020,0.0,0.7,14.0,30,0
46,1.015,0.0,1.2,12.4,21,0
298,1.020,0.0,1.2,13.5,32,0


In [14]:
minmax = MinMaxScaler()
X_train = minmax.fit_transform(X_train)
X_test = minmax.transform(X_test)

In [22]:
import pickle
import os
scaler_path=os.path.join('models/scaler_kidney.pkl')
with open(scaler_path,'wb') as scaler_file:
    pickle.dump(minmax,scaler_file)

In [23]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(criterion='gini',n_estimators=100)
rf.fit(X_train,y_train)

RandomForestClassifier()

In [26]:
from sklearn import model_selection
kfold = model_selection.KFold(n_splits=10)
scoring = 'accuracy'

acc_rf = cross_val_score(estimator = rf, X = X_train, y = y_train, cv = kfold,scoring=scoring)
acc_rf.mean()

0.9954545454545455

In [25]:
import joblib
model_path=os.path.join('models/rf_kidney.sav')
joblib.dump(rf,model_path)

['C:/Users/my pc/Desktop/Stroke-Risk-Prediction-imp/models/rf_kidney.sav']

In [30]:
X.corrwith(y)

sg      0.763203
al     -0.668396
sc     -0.452412
hemo    0.769254
pcv     0.739154
htn    -0.664513
dtype: float64

In [28]:
X


,sg,al,sc,hemo,pcv,htn
0,1.020,1.0,1.2,15.4,28,1
1,1.020,4.0,0.8,11.3,22,0
2,1.010,2.0,1.8,9.6,15,0
3,1.005,4.0,3.8,11.2,16,1
4,1.010,2.0,1.4,11.6,19,0
...,...,...,...,...,...,...
395,1.020,0.0,0.5,15.7,31,0
396,1.025,0.0,1.2,16.5,38,0
397,1.020,0.0,0.6,15.8,33,0
398,1.025,0.0,1.0,14.2,35,0


In [29]:
y

0      0
1      0
2      0
3      0
4      0
      ..
395    1
396    1
397    1
398    1
399    1
Name: classification, Length: 287, dtype: int32